## Deploy R Shiny Dashboard App

### Introduction

Now that we have deployed the machine learning model and WatsonOpenscale has been configured to monitor the model, we can deploy sample R-shiny dashboard in this notebook. 

In this notebook, we will:
* Programmatically get the ID's for the deployment space and model deployment that were created in the **1-model_training** notebook.
* Store Shiny assets into the same deployment space.
* Deploy Shiny assets as an app and view the dashboard

**Sample Materials, provided under license. <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2020. All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp. <br>**

In [1]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

import os
import pandas as pd
import datetime



token = os.environ['USER_ACCESS_TOKEN']
wml_credentials = {
   "token": token,
   "instance_id" : "openshift",
   "url": os.environ['RUNTIME_ENV_APSX_URL'],
   "version": "3.0.0"
}


client = WatsonMachineLearningAPIClient(wml_credentials)

# use this library for reading and saving data in CP4D
from project_lib import Project
project = Project()

### Set up Deployment Space, Deployments and Assets

The following code programmatically gets the deployment space and the model deployment details which were created in `1-model_training` notebook. 
We use the space name specified below. If multiple spaces with the same name exist, the code will take the space that was created most recently. 


Alternatively, the user can manually enter the space name.

In [2]:
space_name = 'Utilities Customer Attrition Space V2'

If the user would like to use a different space manually set the space_id.

Set the space as the default space for working.

In [3]:
l_space_details = []
l_space_details_created_times = []
for space_details in client.spaces.get_details()['resources']:
    if space_details['entity']['name'] == space_name:
        l_space_details.append(space_details)
        l_space_details_created_times.append(datetime.datetime.strptime(space_details['metadata']['created_at'],  '%Y-%m-%dT%H:%M:%S.%fZ'))
        
# get the index of the latest created date from the list and use that to get the space_id
list_latest_index = l_space_details_created_times.index(max(l_space_details_created_times))
space_id = l_space_details[list_latest_index]['metadata']['guid']
# set this space as default space
client.set.default_space(space_id)

'SUCCESS'

# Deploy Shiny App

We will complete the steps to deploy a Shiny Dashboard in Cloud Pak for Data. The app can be deployed in a similar way to models and functions, using the `watson_machine_learning_client` package.

All of the files associated with the dashboard are contained in a zip file which is stored in data assets. If the user would like to make changes to the dashboard, they can download the zip from data assets and upload it in the RStudio IDE. 

In [4]:
r_shiny_deployment_name='Utilities_Customer_Attrition_V2_Shiny_App'

### Store the App

Create the associated metadata and store the dashboard zip file in the deployment space. 

In [9]:
# Meta_props to store assets in space 
meta_props = {
    client.shiny.ConfigurationMetaNames.NAME: "Utilities_Customer_Attrition_Shiny_assets",
    client.shiny.ConfigurationMetaNames.DESCRIPTION: 'Store shiny assets in deployment space' # optional
}
app_details = client.shiny.store(meta_props, '/project_data/data_asset/utilities-customer-attrition-prediction-analytics-dashboard-v2.zip')

Creating Shiny asset...
SUCCESS


### Deploy the App

Create the metadata for the Shiny deployment by providing  name, description, R-Shiny options and Hardware specifications. R-Shiny configuration provides options on whom you want to share the dashboard with, they are 1) anyone with the link 2) Authenticated users 3) Collaborators in this deployment space

In [10]:
# Deployment metadata.
deployment_meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: r_shiny_deployment_name,
    client.deployments.ConfigurationMetaNames.DESCRIPTION: 'Deploy Utilities Customer Attrition dashboard',
    client.deployments.ConfigurationMetaNames.R_SHINY: { 'authentication': 'anyone_with_url' },
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { 'name': 'S', 'num_nodes': 1}
}

# Create the deployment.
app_uid = client.shiny.get_uid(app_details)
rshiny_deployment = client.deployments.create(app_uid, deployment_meta_props)



#######################################################################################

Synchronous deployment creation for uid: '4a9d7f61-b213-4d57-823e-8851b364b543' started

#######################################################################################


initializing......
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='ba40b3d6-63bc-4f40-b8e5-d0dd0d9e1ffd'
------------------------------------------------------------------------------------------------




### Launch Shiny App
Now that the dashboard is deployed, it can be accessed through the web browser. The app URL can be found by navigating to the deployed app in the deployment space. 

Open the Navigation Menu, under ***Analytics*** select ***Analytics deployments -> Utilities Customer Attrition Space -> Deployments -> Utilities_Customer_Attrition_Shiny_App*** to find the dashboard URL.

Alternatively, the path for the app URL can be found from the deployment metadata created in the previous cell. This path should be appended to the user's Cloud Pak for Data hostname to get the complete app URL. To get the path, run the cell below:

In [11]:
print("{HOSTNAME}"+rshiny_deployment['metadata']['href'] + '/r_shiny')

{HOSTNAME}/v4/deployments/ba40b3d6-63bc-4f40-b8e5-d0dd0d9e1ffd/r_shiny
